In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from scripts.entity_utils import *
import pandas as pd
import seaborn as sns

In [ ]:
from dygie.models.global_analysis.clustering import *
from dygie.models.global_analysis.relation_extraction import *

In [ ]:
from dygie.training.evaluation import *
dev_data = [json.loads(line) for line in open('../model_data/pwc_split_on_sectioned/dev.jsonl')]
cluster_matching_thresholds = generate_thresholds(dev_data)
print(cluster_matching_thresholds)

In [ ]:
dev_pred = [json.loads(line) for line in open('../outputs/test_results_folder/combined.jsonl')]

In [ ]:
def add_type_to_gold(x) :
    x['n_ary_relations'] = [Relation(*y)._asdict() for y in x['n_ary_relations']]
    x['true_entities'] = {}
    for entity in used_entities :
        x['true_entities'][entity] = list(set([rel[entity] for rel in x['n_ary_relations']]))
        
    x['gold_to_type'] = {}
    for rel in x['n_ary_relations'] :
        for k, v in rel.items() :
            x['gold_to_type'][v] = k
            
    x['gold_clusters'] = {k:{'spans' : v, 'words' : [" ".join(x['words'][y[0]:y[1]]) for y in v]} for k, v in x['coref'].items()}
            
    return x

In [ ]:
dev_pred = [json.loads(line) for line in open('../outputs/test_results_folder/combined.jsonl')]
dev_data = [json.loads(line) for line in open('../model_data/pwc_split_on_sectioned/test.jsonl')]

dev_data = {x['doc_id']:x for x in dev_data}
for x in dev_pred :
    x['true_coref'] = dev_data[x['doc_id']]['coref']
    x['n_ary_relations'] = [Relation(*y)._asdict() for y in dev_data[x['doc_id']]['n_ary_relations']]
    x['true_entities'] = {}
    for entity in used_entities :
        x['true_entities'][entity] = list(set([rel[entity] for rel in x['n_ary_relations']]))
        
    x['gold_to_type'] = {}
    for rel in x['n_ary_relations'] :
        for k, v in rel.items() :
            x['gold_to_type'][v] = k

In [ ]:
for d in dev_pred :
    for k in d['true_coref'] :
        if len(d['true_coref'][k]) == 0 :
            print(d['doc_id'], k, d['true_coref'][k])

In [ ]:
hand_annotation = {'007ff2ca5f297b04636699ce4d01ca6d6f21dc77' : { 'table' : ['Matched', 'Mismatched'] },
                  '01c824989d24a8cae214c3156edd9d4492faa579' : { 'missed' : ['Conditional_Image_Generation']},
                  '027d6c52be01b583b9a0d9eb8c9364c6b701c656' : { 'missed' : ['Percentage_correct']},
                  '05d2700846c0323f79c1344aca5333994c7c03a5' : { 'missed' : ['IBM_2016']},
                  '06c06885fd53b2cbd407704cf14f658842ed48e5' : { 'missed' : ['Set14_-_4x_upscaling', 'Set5_-_4x_upscaling']},
                  '07a9478e87a8304fc3267fa16e83e9f3bbd98b27' : { 'missed' : ['Parameters']},
                  '0910a4c470a410fac446f4026f7c8ef512ae7427' : { 'missed' : ['Percentage_correct']},
                  '0a78873e41615798d09391d9f40d41666b8c9beb' : { 'missed' : ['Participant_Intervention_Comparison_Outcome_Extraction']}}

In [ ]:
for v in dev_pred :
    v['missed'] = hand_annotation.get(v['doc_id'], {}).get('missed', [])
    v['table'] = hand_annotation.get(v['doc_id'], {}).get('table', [])

In [ ]:
for v in dev_pred :
    v['prediction'] = {tuple(x['span']):x['label'] for x in v['prediction']}
    v['gold'] = {tuple(x['span']):x['label'] for x in v['gold']}

In [ ]:
for ex in dev_pred :
    clusters, spl, cluster_labels = do_clustering(ex, 'prediction', 'coref_prediction', plot=True)
    ex['clusters'] = clusters
    ex['linked_clusters'] = get_linked_clusters(clusters)
    relation_matrix = get_relations_between_clusters(ex, 'prediction', cluster_labels) #, ex['relation_threshold'])
    ex['relation_matrix'] = relation_matrix
    ex['predicted_relations'] = {n:generate_relations(clusters, relation_matrix, linked_clusters, n, ex['relation_threshold'])
                                 for n in range(1, 6)}
    
    ex['gold_clusters'] = {k:{'spans' : v, 'words' : [" ".join(ex['words'][y[0]:y[1]]) for y in v]} for k, v in ex['true_coref'].items()}
    ex['true_relations'] = {n:generate_true_relations(ex['n_ary_relations'], n)
                                 for n in range(1, 6)}
    
    map_all_clusters_to_true(ex, cluster_matching_thresholds)

In [ ]:
p = 0
for n in range(len(dev_pred)) :
#     print(dev_pred[n]['linked_clusters'])
    for rel in [Relation(**x) for x in dev_pred[n]['n_ary_relations']] :
        print(rel)
    print("-"*30)
    for v in dev_pred[n]['predicted_relations'][4].values() :
        p += len(set(v))
        for x in set(v) :
            print([dev_pred[n]['clusters'][k]['matched'] if dev_pred[n]['clusters'][k]['matched'] is not None else dev_pred[n]['clusters'][k]['name']
                   for k in x])

    print("="*30)
print(p)

In [ ]:
# print(evaluate_for_all_document(dev_pred))

oracle_res, oracle_exp_res, oracle_exp_res_table, gold_res, pred_res = run_eval_for_relation_extraction(dev_pred)
x = []
p = []
g = []
o = []
oe = []
oet = []
oa = []

for n, v in gold_res.items() :
    if n < 5 :
        print(n)
        x.append(n)
        print(pd.DataFrame(v))
        g.append((pd.DataFrame(v).mean(1)*100).to_dict())
        g[-1].update({"n" : str(n)})
        print('-'*20)
        print(pd.DataFrame(pred_res[n]))
        p.append((pd.DataFrame(pred_res[n]).mean(1)*100).to_dict())
        p[-1].update({"n" : str(n)})
        print("="*20)
        print(pd.DataFrame(oracle_res[n]))
        o.append((pd.DataFrame(oracle_res[n]).mean(1)*100).to_dict())
        o[-1].update({"n" : str(n)})
        print("="*20)
        oe.append((pd.DataFrame(oracle_exp_res[n]).mean(1)*100).to_dict())
        oe[-1].update({"n" : str(n)})
        oet.append((pd.DataFrame(oracle_exp_res_table[n]).mean(1)*100).to_dict())
        oet[-1].update({"n" : str(n)})
        oa.append((pd.DataFrame(oracle_abs[n]).mean(1)*100).to_dict())
        oa[-1].update({"n" : str(n)})

In [ ]:
p

In [ ]:
data = pd.DataFrame(p)
data['type'] = 'predicted relation'

data_1 = pd.DataFrame(g)
data_1['type'] = 'gold relation'

data_2 = pd.DataFrame(o)
data_2['type'] = 'oracle relation'

data_3 = pd.DataFrame(oe)
data_3['type'] = 'oracle (expert) relation'

data_4 = pd.DataFrame(oet)
data_4['type'] = 'oracle (expert/table) relation'

# data_5 = pd.DataFrame(oa)
# data_5['type'] = 'abstract only'
# data_5['f1'] = [33.1, 11.6, 3.4, 0.0]

data = pd.concat([data, data_1, data_2, data_3, data_4]) #, data_5])

sns.lineplot(x='n', y='f1', hue='type', data=data)
plt.scatter(x=[3], y=[24.6], label="Manual Evaluation")
plt.scatter(x=[3], y=[8.69], label="Oracle (Abstract Only) Relation", c='saddlebrown')
plt.ylabel("F1-score")
plt.xlabel("N-ary Relation")
plt.gca().legend(loc='center left', bbox_to_anchor=(1, 0.5))
sns.despine()

In [ ]:
# from scripts.convert_predictions_to_brat import *
# generate_brat_annotations(documents={x['doc_id']:x for x in dev_pred}, 
#                           brat_anno_folder='../brat/data/result_extraction_results/ner')

In [ ]:
from scripts.result_from_pwc_table import *

In [ ]:
map_to_arxiv = json.load(open('../data/arxiv_map.json'))

In [ ]:
for x in dev_pred :
    x['arxiv_id'] = map_to_arxiv[x['doc_id']]

In [ ]:
for x in dev_pred :
    if x['arxiv_id'] is not None :
        print(x['doc_id'])
        clusters = [x['words'] for x in x['gold_clusters'].values()]
        print(get_scores(x['arxiv_id'], clusters))

In [ ]:
# train_data = [add_type_to_gold(json.loads(line)) for line in open('../model_data/pwc_split_on_sectioned/train.jsonl')]
# train_data += [add_type_to_gold(json.loads(line)) for line in open('../model_data/pwc_split_on_sectioned/dev.jsonl')]
train_data = [add_type_to_gold(json.loads(line)) for line in open('../model_data/pwc_split_on_sectioned/test.jsonl')]

In [ ]:
for d in train_data :
    map_gold_clusters_to_true(d, cluster_matching_thresholds)

In [ ]:
from collections import defaultdict
mentions_in_doc = {k:defaultdict(int) for k in used_entities}
matched_in_doc = {k:defaultdict(int) for k in used_entities}
for ins in train_data :
    for v, e in ins['gold_to_type'].items() :
        if e in mentions_in_doc :
            l = len(ins['gold_clusters'][v]['spans'])
            mentions_in_doc[e][l if l < 1 else 1] += 1
            
            l1 = ins['gold_clusters'][v]['matched']
            matched_in_doc[e][1 if l1 is not None else 0] += 1
            if l > 0 and l1 is None :
                print(v, ins['gold_clusters'][v])

In [ ]:
for e, v in mentions_in_doc.items() :
    print(e, v[1] / sum(v.values()))

In [ ]:
for e, v in matched_in_doc.items() :
    print(e, v[1] / sum(v.values()))

In [ ]:
n_gold, n_present = [], []
for ins in train_data :
    n_gold.append(len(ins['n_ary_relations']))
    n_present.append(sum([all([len(ins['coref'][v]) > 0 for k, v in rel.items() if k != 'score']) for rel in ins['n_ary_relations']]))

In [ ]:
plt.scatter(n_gold, n_present)

In [ ]:
sum(n_present) / sum(n_gold)

In [ ]:
import seaborn as sns
x = ["0" if k == 0 else " >0" for e in mentions_in_doc for k, v in mentions_in_doc[e].items()]
y = [v for e in mentions_in_doc for k, v in mentions_in_doc[e].items()]
h = [e for e in mentions_in_doc for k, v in mentions_in_doc[e].items()]
sns.barplot(x, y, h)

In [ ]:
length = []
for d in train_data :
    for c, spans in d['coref'].items() :
        for s in spans :
            length.append(s[1] - s[0])

In [ ]:
sns.countplot(length)

In [ ]:
sum(np.array(length) > 10)

In [ ]:
plt.plot(np.linspace(1, 5, 5), 100 * (1 - 0.2)**np.linspace(1, 5, 5))
plt.plot(np.linspace(1, 5, 5), 100 * (1 - 0.4)**np.linspace(1, 5, 5))
plt.plot(np.linspace(1, 5, 5), 100 * (1 - 0.6)**np.linspace(1, 5, 5))
plt.plot(np.linspace(1, 5, 5), 100 * (1 - 0.8)**np.linspace(1, 5, 5))

In [ ]:
dev_data[0]